In [ ]:
# run IPYNB that imports all the relevant packages and functions
%run ../ms_packages_and_functions.ipynb

In [ ]:
output_dir = pjoin(os.getcwd(), 'processed_data')
data_dir   = pjoin(os.getcwd(), 'raw_data')
fig_dir    = pjoin(os.getcwd(), 'figures')

In [ ]:
data = loadPickle('processed_data/rj1031209_CenterOut_data_10msBin_with500buffer_MIonly.pkl')

In [ ]:
rand_state = '003' #np.random.randint(1000000000)

In [ ]:
folder_path = pjoin(output_dir,'RJ_nulls-poisson-200ms-{}'.format(rand_state))

if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    print("created directory: {}".format(folder_path))
else:
    print("All files will be saved in: {}".format(folder_path))


In [ ]:
num_processes = 10  # how many parallel processes to run

In [ ]:
buffer = 0.5
kernel_sigma = 20*pq.ms
sampling_period = 10*pq.ms
kernel = el.kernels.GaussianKernel(sigma = kernel_sigma)

mov_window = 0.2 # 200 ms
binwin = data[0]['binwin']
mov_10ms = int(0.01/binwin)
windowBin = int(mov_window/binwin)

num_units = data[0]['trialData'][0]['trialNumUnits']

In [ ]:
run = True
num_sim = 15

In [ ]:
if run:
    for sim in range(0,num_sim):
        poisson_rasters = []
        poisson_FNs = []

        direction_rasters = []
        direction_FNs = []
        for direction in range(0,len(data)):

            windowBin = int(mov_window/binwin)
            direction_rasters.append([])
            direction_FNs.append([])

            trial_rasters  = []
            trial_FNs = []
            
            for trial in range(0,len(data[direction]['trialData'])):
                trial_rasters.append([])
                trial_FNs.append([])

                poiss = {}
                poiss['spiketimes'] = []

                # start time at 0
                # bx has 500ms buffer in the trial start and end
                t_start = data[direction]['trialData'][trial]['trialStart'] - buffer
                t_stop  =  data[direction]['trialData'][trial]['trialEnd'] + buffer - t_start
                

                for unit in range(0,num_units):
                    spiketimes = np.array(data[direction]['trialData'][trial]['trialSpiketimes']['spiketimes'][unit])-t_start
                    spiketrain = neo.core.SpikeTrain(spiketimes*pq.s,\
                                    t_start = 0,\
                                    t_stop = t_stop)
                    fr = el.statistics.instantaneous_rate(spiketrain,sampling_period=sampling_period,kernel=kernel)
                    
                    poiss_spiketrain = el.spike_train_generation.inhomogeneous_poisson_process(fr)

                    poiss['spiketimes'].append(poiss_spiketrain.times.simplified.magnitude)
                    
                trialData,_ = bin_spikes(poiss,0,t_stop,binwin,plot=False)
                
                test = [[x[j:j+windowBin] for x in trialData] for j in range(0,trialData[0].shape[0]-windowBin,mov_10ms)]

                status("> Generating FN for direction: {}/{},trial: {}/{}".format(direction, len(data), \
                            trial,len(data[direction]['trialData'])))

                with Pool(num_processes) as p:
                    time_FNs = p.map(make_FN, test)

                trial_FNs[trial].append(time_FNs)
            #     trial_rasters[trial].append(trialData)

            # direction_rasters[direction].append(trialData)
            direction_FNs[direction].append(trial_FNs)

        # poisson_rasters.append(direction_rasters)
        poisson_FNs.append(direction_FNs)

        # print('> SAVING...')
        # save_dir = pjoin(folder_path,'poisson_rasters_{}.pkl'.format(str(sim).zfill(3)))
        # savePickle(save_dir,poisson_rasters)
        # print('>> saved poisson_rasters in folder: {}'.format(save_dir))
        

        save_dir = pjoin(folder_path,'poisson_FNs_{}.pkl'.format(str(sim).zfill(3)))
        savePickle(save_dir,poisson_FNs)
        print('>> saved poisson_FNs in folder: {}'.format(save_dir))

        # del poisson_rasters,poisson_FNs, direction_rasters,   direction_FNs, trial_rasters, trial_FNs
        del poisson_FNs,direction_FNs, trial_FNs


In [ ]:
# save_dir = pjoin(folder_path,'poisson_FNs_{}.pkl'.format(str(sim).zfill(3)))
# savePickle(save_dir,poisson_FNs)
# print('>> saved poisson_FNs in folder: {}'.format(save_dir))